## 数据背景

#### 本数据为一家银行的个人金融业务数据集，可以作为银行场景下进行 个人客户业务分析和数据挖掘的示例。这份数据中涉及到5300个银行 客户的100万笔的交易，而且涉及700份贷款信息与近900张信用卡的数 据。通过分析这份数据可以获取与银行服务相关的业务知识。例如， 提供增值服务的银行客户经理，希望明确哪些客户有更多的业务需求， 而风险管理的业务人员可以及早发现贷款的潜在损失。
#### 可否根据客户贷款前的属性、状态信息和交易行为预测其贷款违约行为?

## 数据含义

#### 名称      标签         说明    
#### disp_id  权限号
#### loan_id  贷款号 (主键)
#### account_ id账户号
#### date     发放贷款日期
#### amount   贷款金额
#### duration 贷款期限
#### payments 每月归还额
#### status   还款状态     A代表合同终止.没问题;B代表合同终止,贷款没有支付;C代表合同处于执行期,至今正常;D代表合同处于执行期,欠债状态。

## 分析数据

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.getcwd()     #查看当前工作目录

'/Users/jvf/Downloads/数据分析/项目/个人贷款违约预测模型/个人'

In [3]:
loanfile = os.listdir()
loanfile
createVar = locals()
for i in loanfile :
    if i.endswith('csv'):
        createVar[i.split('.')[0]]=pd.read_csv(i,encoding='gbk')   
        print (i.split('.')[0])

district
clients
accounts
loans
trans
card
order
disp


/Users/jvf/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#####   loans        : 贷款表
#####  accounts ：账户表
#####  disp         ：权限分配表
#####  clients    ：客户信息表     每条记录代表客户和账户之间的关系，及客户操作账户的权限
#####  order     ：支付命令表      每条记录代表一个支付交易命令
#####  trans     ：支付命令表      每条记录代表每个账户每一笔交易记录，1056210条记录
#####  district   ：人口地区统计表      每条记录代表每个地区人口统计信息，GDP等
#####  card      ：信用卡信息表      每条记录代表每个账户信用卡信息

### 2.1 定义信用评级

In [4]:
loans['status'].value_counts()

C    403
A    203
D     45
B     31
Name: status, dtype: int64

In [5]:
bad_good = {'A':0,'B':1,'D':1,'C':2}   
loans['bad_good']=loans.status.map(bad_good)

#####  A代表合同终止.没问题;B代表合同终止,贷款没有支付;C代表合同处于执行期,至今正常;D代表合同处于执行期,欠债状态。

##### map()是 Python 内置的高阶函数，它接收一个函数 f 和一个 list，并通过把函数 f 依次作用在 list 的每个元素上，得到一个新的 list 并返回。

In [6]:
loans.head()

,loan_id,account_id,date,amount,duration,payments,status,bad_good
0,5314,1787,1993-07-05,96396,12,8033,B,1
1,5316,1801,1993-07-11,165960,36,4610,A,0
2,6863,9188,1993-07-28,127080,60,2118,A,0
3,5325,1843,1993-08-03,105804,36,2939,A,0
4,7240,11013,1993-09-06,274740,60,4579,A,0


### 2.2 合并表格

#### 2.2.1合并loans表和disp表

In [7]:
disp.head()    #对比disp表和Loans表，共同存在account_id列

,disp_id,client_id,account_id,type
0,1,1,1,所有者
1,2,2,2,所有者
2,3,3,2,用户
3,4,4,3,所有者
4,5,5,3,用户


In [8]:
data2 = pd.merge(loans,disp,on='account_id',how='left')   #合并两张表
data2.shape[0]   #查看dataframe行数

827

In [9]:
data2.head()

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,type
0,5314,1787,1993-07-05,96396,12,8033,B,1,2166,2166,所有者
1,5316,1801,1993-07-11,165960,36,4610,A,0,2181,2181,所有者
2,6863,9188,1993-07-28,127080,60,2118,A,0,11006,11314,所有者
3,5325,1843,1993-08-03,105804,36,2939,A,0,2235,2235,所有者
4,7240,11013,1993-09-06,274740,60,4579,A,0,13231,13539,所有者


#### 2.2.2合并data2表和clients表

In [10]:
clients.head()    #对比data2表和clients表，共同存在client_id列

,client_id,sex,birth_date,district_id
0,1,女,1970-12-13,18
1,2,男,1945-02-04,1
2,3,女,1940-10-09,1
3,4,男,1956-12-01,5
4,5,女,1960-07-03,5


In [11]:
data3 = pd.merge(data2,clients,on='client_id',how='left')
data3.head()

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,type,sex,birth_date,district_id
0,5314,1787,1993-07-05,96396,12,8033,B,1,2166,2166,所有者,女,1947-07-22,30
1,5316,1801,1993-07-11,165960,36,4610,A,0,2181,2181,所有者,男,1968-07-22,46
2,6863,9188,1993-07-28,127080,60,2118,A,0,11006,11314,所有者,男,1936-06-02,45
3,5325,1843,1993-08-03,105804,36,2939,A,0,2235,2235,所有者,女,1940-04-20,14
4,7240,11013,1993-09-06,274740,60,4579,A,0,13231,13539,所有者,男,1978-09-07,63


In [12]:
district.head()  #A1列=district_id列，指地区号

,A1,GDP,A4,A10,A11,A12,A13,A14,A15,a16
0,1,283894,1204953,100.0,12541,0.29,0.43,167,35.6,41.1
1,2,11655,88884,46.7,8507,1.67,1.85,132,12.1,15.0
2,3,13146,75232,41.7,8980,1.95,2.21,111,18.8,18.7
3,4,16108,149893,67.4,9753,4.64,5.05,109,17.5,19.7
4,5,13452,95616,51.4,9307,3.85,4.43,118,13.7,15.9


#### 2.2.3合并data3,district表

In [13]:
data4 = pd.merge(data3,district,left_on='district_id',right_on='A1',how='left')
data4.head()
#data4[['district_id','A1']].head(3)

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,...,A1,GDP,A4,A10,A11,A12,A13,A14,A15,a16
0,5314,1787,1993-07-05,96396,12,8033,B,1,2166,2166,...,30,16979,94812,81.8,9650,3.38,3.67,100,15.7,14.8
1,5316,1801,1993-07-11,165960,36,4610,A,0,2181,2181,...,46,14111,112709,73.5,8369,1.79,2.31,117,12.7,11.6
2,6863,9188,1993-07-28,127080,60,2118,A,0,11006,11314,...,45,12888,77917,53.5,8390,2.28,2.89,132,13.3,13.6
3,5325,1843,1993-08-03,105804,36,2939,A,0,2235,2235,...,14,31891,177686,74.8,10045,1.42,1.71,135,18.6,17.7
4,7240,11013,1993-09-06,274740,60,4579,A,0,13231,13539,...,63,11322,86513,50.5,8288,3.79,4.52,110,9.0,8.4


#### 2.2.4合并loans表和trans表
##### 求贷款前一年账户的平均余额，余额的标准差，平均收入和平均支出比例

In [14]:
loans.sort_values(['account_id','date']).head()   #贷款表  ，date:发放贷款的日期

,loan_id,account_id,date,amount,duration,payments,status,bad_good
20,4959,2,1994-01-05,80952,24,3373,A,0
239,4961,19,1996-04-29,30276,12,2523,B,1
505,4962,25,1997-12-08,30276,12,2523,A,0
663,4967,37,1998-10-14,318480,60,5308,D,1
582,4968,38,1998-04-19,110736,48,2307,C,2


In [15]:
trans.sort_values('account_id').head()   #交易表：每条记录代表每个账户上的一条记录

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
280771,209,1,1996-02-17,贷,信贷资金,$100,$14907,NaN,NaN,NaN
655184,184,1,1997-09-30,借,支取现金,$15,$16752,支票,NaN,NaN
717942,37,1,1997-12-13,贷,从他行收款,"$3,679",$14749,NaN,AB,41403269.0
475595,114,1,1997-01-30,借,支取现金,$840,$12777,NaN,NaN,NaN
409957,3530456,1,1996-10-31,贷,NaN,$73,$16798,利息所得,NaN,NaN


##### account_id:账户号  type: 借贷类型   operation:交易类型      balance:账户余额   date :交易日期

In [16]:
data5 = pd.merge(loans[['account_id','date']],trans[['account_id','type','amount','balance','date']],on='account_id')
data5.columns = ['account_id','date','type','amount','balance','t_date']
data5.sort_values(['account_id','t_date'])
data5['date']=pd.to_datetime(data5['date'])
data5['t_date']=pd.to_datetime(data5['t_date'])
data5.head(10)


,account_id,date,type,amount,balance,t_date
0,1787,1993-07-05,贷,"$1,100","$1,100",1993-03-22
1,1787,1993-07-05,贷,"$9,900",$11000,1993-04-21
2,1787,1993-07-05,贷,"$5,800",$16800,1993-05-21
3,1787,1993-07-05,贷,"$3,300",$20100,1993-06-20
4,1787,1993-07-05,贷,$42248,$62348,1993-07-08
5,1787,1993-07-05,借,$16600,$45748,1993-07-20
6,1787,1993-07-05,贷,$74,$45822,1993-07-31
7,1787,1993-07-05,借,$12400,$33422,1993-08-19
8,1787,1993-07-05,贷,$191,$33612,1993-08-31
9,1787,1993-07-05,借,"$3,300",$30312,1993-09-18


#### 清洗数据

对账户余额进行清洗,去掉$符号

In [17]:
data5['balance2']=data5['balance'].map(lambda x : int (''.join(x[1:].split(','))))
data5['amount2']=data5['amount'].map(lambda x : int (''.join(x[1:].split(','))))

In [18]:
data5.head()

,account_id,date,type,amount,balance,t_date,balance2,amount2
0,1787,1993-07-05,贷,"$1,100","$1,100",1993-03-22,1100,1100
1,1787,1993-07-05,贷,"$9,900",$11000,1993-04-21,11000,9900
2,1787,1993-07-05,贷,"$5,800",$16800,1993-05-21,16800,5800
3,1787,1993-07-05,贷,"$3,300",$20100,1993-06-20,20100,3300
4,1787,1993-07-05,贷,$42248,$62348,1993-07-08,62348,42248


#### 只取贷款日期前一年的交易记录

In [19]:
import datetime
data6 =data5[data5.date>data5.t_date][data5.date<data5.t_date+datetime.timedelta(days=365)]

/Users/jvf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [20]:
data6.head()

,account_id,date,type,amount,balance,t_date,balance2,amount2
0,1787,1993-07-05,贷,"$1,100","$1,100",1993-03-22,1100,1100
1,1787,1993-07-05,贷,"$9,900",$11000,1993-04-21,11000,9900
2,1787,1993-07-05,贷,"$5,800",$16800,1993-05-21,16800,5800
3,1787,1993-07-05,贷,"$3,300",$20100,1993-06-20,20100,3300
403,1801,1993-07-11,贷,$700,$700,1993-02-13,700,700


#### 求账户余额，账户余额标准差，变异系数

In [21]:
data7 = data6.groupby('account_id')['balance2'].agg([('avg_balance','mean'),('std_balance','std')]) #账户余额和账户标准差

In [22]:
data7['cv_balance'] = data7[['avg_balance','std_balance']].apply(lambda x :x[1]/x[0],axis=1)   #变异系数

##### 变异系数=标准差/账户余额   ，变异系数越大，说明经济状况越不稳定

In [23]:
data7.head()

,avg_balance,std_balance,cv_balance
account_id,,,
2,32590.759259,12061.802206,0.370099
19,25871.223684,15057.521648,0.582018
25,56916.984496,21058.667949,0.369989
37,36658.981308,20782.996690,0.566928
38,31383.581818,10950.723180,0.348932


### 求收入支出比

In [24]:
type_dict = {'借':'out','贷':'income'}
data6['type2']=data6.type.map(type_dict)

In [25]:
data8 = data6.groupby(['account_id','type2'])[['amount2']].sum()

In [26]:
data8.head()

amount2
account_id type2          
2          income   276514
           out      153020
19         income   254255
           out      198020
25         income   726479

In [27]:
data9 = pd.pivot_table(data8,values='amount2',index='account_id',columns='type2') #将该数组转置
data9['out/in'] = data9.apply(lambda x : x[1]/x[0],axis=1)      #求支出收入比，支出/收入

In [28]:
data9.head()

type2,income,out,out/in
account_id,,,
2,276514.0,153020.0,0.553390
19,254255.0,198020.0,0.778824
25,726479.0,629108.0,0.865969
37,386357.0,328541.0,0.850356
38,154300.0,105091.0,0.681082


### 合并为总表

In [29]:
data = pd.merge(data7,data9,on='account_id',how='left')
data = pd.merge(data4,data,on='account_id',how='left')
data=data.sort_values('account_id')

In [30]:
data.head()

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,...,A13,A14,A15,a16,avg_balance,std_balance,cv_balance,income,out,out/in
23,4959,2,1994-01-05,80952,24,3373,A,0,3,3,...,0.43,167,35.6,41.1,32590.759259,12061.802206,0.370099,276514.0,153020.0,0.553390
22,4959,2,1994-01-05,80952,24,3373,A,0,2,2,...,0.43,167,35.6,41.1,32590.759259,12061.802206,0.370099,276514.0,153020.0,0.553390
289,4961,19,1996-04-29,30276,12,2523,B,1,25,25,...,2.07,123,11.1,11.4,25871.223684,15057.521648,0.582018,254255.0,198020.0,0.778824
615,4962,25,1997-12-08,30276,12,2523,A,0,31,31,...,4.72,96,12.3,12.9,56916.984496,21058.667949,0.369989,726479.0,629108.0,0.865969
806,4967,37,1998-10-14,318480,60,5308,D,1,45,45,...,3.64,120,11.1,10.9,36658.981308,20782.996690,0.566928,386357.0,328541.0,0.850356


#### 求贷存比和代收比

In [31]:
data['r_lb'] = data[['amount','avg_balance']].apply(lambda x : x[0]/x[1] ,axis=1)
data['r_lincome'] =data[['amount','income']].apply(lambda x: x[0]/x[1],axis=1)

In [32]:
data.head()

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,...,A15,a16,avg_balance,std_balance,cv_balance,income,out,out/in,r_lb,r_lincome
23,4959,2,1994-01-05,80952,24,3373,A,0,3,3,...,35.6,41.1,32590.759259,12061.802206,0.370099,276514.0,153020.0,0.553390,2.483894,0.292759
22,4959,2,1994-01-05,80952,24,3373,A,0,2,2,...,35.6,41.1,32590.759259,12061.802206,0.370099,276514.0,153020.0,0.553390,2.483894,0.292759
289,4961,19,1996-04-29,30276,12,2523,B,1,25,25,...,11.1,11.4,25871.223684,15057.521648,0.582018,254255.0,198020.0,0.778824,1.170258,0.119077
615,4962,25,1997-12-08,30276,12,2523,A,0,31,31,...,12.3,12.9,56916.984496,21058.667949,0.369989,726479.0,629108.0,0.865969,0.531933,0.041675
806,4967,37,1998-10-14,318480,60,5308,D,1,45,45,...,11.1,10.9,36658.981308,20782.996690,0.566928,386357.0,328541.0,0.850356,8.687639,0.824315


### 建立模型

#### 查看数据列名

In [33]:
data.columns

Index(['loan_id', 'account_id', 'date', 'amount', 'duration', 'payments',
       'status', 'bad_good', 'disp_id', 'client_id', 'type', 'sex',
       'birth_date', 'district_id', 'A1', 'GDP', 'A4', 'A10', 'A11', 'A12',
       'A13', 'A14', 'A15', 'a16', 'avg_balance', 'std_balance', 'cv_balance',
       'income', 'out', 'out/in', 'r_lb', 'r_lincome'],
      dtype='object')

#### 创建训练集和测试集

In [34]:
data_model = data[data.status != 'C']  #C代表合同处于执行期,至今正常。
for_predict = data[data.status == 'C']

In [35]:
#data_model.shape[0]
#for_predict.shape[0]

In [36]:
train = data_model.sample(frac=0.7,random_state=1235).copy()
test = data_model[~data_model.index.isin(train.index)].copy()
print('样本数量：%i \n训练样本数量：%i \n测试样本数量：%i'%(len(data_model),len(train),len(test)))

样本数量：334 
训练样本数量：234 
测试样本数量：100


In [37]:
#train.head()
test.head()

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,...,A15,a16,avg_balance,std_balance,cv_balance,income,out,out/in,r_lb,r_lincome
289,4961,19,1996-04-29,30276,12,2523,B,1,25,25,...,11.1,11.4,25871.223684,15057.521648,0.582018,254255.0,198020.0,0.778824,1.170258,0.119077
611,4988,103,1997-12-06,265320,36,7370,D,1,124,124,...,10.3,11.0,46666.109589,27699.868290,0.593576,434901.0,243539.0,0.559987,5.685496,0.610070
377,4996,132,1996-11-06,88440,12,7370,A,0,158,158,...,27.1,26.8,59552.290323,22826.656986,0.383304,284975.0,175400.0,0.615493,1.485081,0.310343
337,5088,544,1996-08-22,91152,24,3798,A,0,655,655,...,20.7,21.4,45932.779412,12574.796275,0.273765,335055.0,274250.0,0.818522,1.984465,0.272051
543,5117,718,1997-08-20,76944,12,6412,A,0,861,861,...,35.6,41.1,39164.323232,21030.225397,0.536974,472143.0,368107.0,0.779652,1.964645,0.162968


### 建模(使用逻辑回归模型)

#### 向前逐步法

In [38]:
def forward_select(data, response):
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    while remaining:
        aic_with_candidates=[]
        for candidate in remaining:
            formula = "{} ~ {}".format(
                response,' + '.join(selected + [candidate]))
            aic = smf.glm(
                formula=formula, data=data, 
                family=sm.families.Binomial(sm.families.links.logit)
            ).fit().aic
            aic_with_candidates.append((aic, candidate))
        aic_with_candidates.sort(reverse=True)
        best_new_score, best_candidate=aic_with_candidates.pop()
        if current_score > best_new_score: 
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
            print ('aic is {},continuing!'.format(current_score))
        else:        
            print ('forward selection over!')
            break
            
    formula = "{} ~ {} ".format(response,' + '.join(selected))
    print('final formula is {}'.format(formula))
    model = smf.glm(
        formula=formula, data=data, 
        family=sm.families.Binomial(sm.families.links.logit)
    ).fit()
    return(model)

#### 选择特征

In [39]:
data.columns

Index(['loan_id', 'account_id', 'date', 'amount', 'duration', 'payments',
       'status', 'bad_good', 'disp_id', 'client_id', 'type', 'sex',
       'birth_date', 'district_id', 'A1', 'GDP', 'A4', 'A10', 'A11', 'A12',
       'A13', 'A14', 'A15', 'a16', 'avg_balance', 'std_balance', 'cv_balance',
       'income', 'out', 'out/in', 'r_lb', 'r_lincome'],
      dtype='object')

In [40]:
candidates = ['bad_good', 'A1', 'GDP', 'A4', 'A10', 'A11', 'A12','amount', 'duration',
       'A13', 'A14', 'A15', 'a16', 'avg_balance', 'std_balance',
       'cv_balance', 'income', 'out', 'out/in', 'r_lb', 'r_lincome']
data_for_select = train[candidates]
lg_m1 = forward_select(data=data_for_select,response='bad_good')
lg_m1.summary().tables[1]

AssertionError: 

In [ ]:
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
fpr, tpr, th = metrics.roc_curve(test.bad_good, lg_m1.predict(test))
plt.figure(figsize=[6, 6])
plt.plot(fpr, tpr, 'b--')
plt.title('ROC curve')
plt.show()

In [ ]:
print('AUC = %.4f' %metrics.auc(fpr, tpr))

In [ ]:
for_predict['prob']=lg_m1.predict(for_predict)
for_predict[['account_id','prob']].head()